In [1]:
import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K

import numpy as np

import h5py

import tensorflow as tf

import utils
reload(utils)

%matplotlib inline

Using TensorFlow backend.


In [2]:
batch_size = 10

data_root = '../data/tu-berlin/sample50/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
#         target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
#         target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 2650 images belonging to 50 classes.
Found 1350 images belonging to 50 classes.
256 256 1 50


In [3]:
model_name = 'sketchnet3'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)

In [4]:
def build_model():
    model = Sequential()
    model.add(Convolution2D(64, (3, 3), 
                            padding='same',
                            activation='relu',
                            input_shape=(img_width, img_height, channels)))
    model.add(Convolution2D(64, (3, 3), 
                            padding='same',
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(128, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(Convolution2D(128, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))

    model.add(Convolution2D(256, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(Convolution2D(256, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # initialize optimizer
    opt = rmsprop(lr=0.0001)

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])
    
    return model, opt

model, opt = build_model()


In [5]:
model.load_weights('../models/sketchnet3/latest.h5')

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 128)       0         
__________

In [8]:
total_epochs=22

In [9]:
nb_epoch = 5
K.set_value(opt.lr, 0.0001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 23/27
265/265 [==============================] - 270s - loss: 3.9123 - acc: 0.0174 - val_loss: 3.9121 - val_acc: 0.0200
Epoch 24/27
265/265 [==============================] - 249s - loss: 3.9122 - acc: 0.0181 - val_loss: 3.9120 - val_acc: 0.0185
Epoch 25/27
265/265 [==============================] - 249s - loss: 3.9122 - acc: 0.0196 - val_loss: 3.9120 - val_acc: 0.0185
Epoch 26/27
265/265 [==============================] - 268s - loss: 3.9122 - acc: 0.0158 - val_loss: 3.9121 - val_acc: 0.0207
Epoch 27/27
265/265 [==============================] - 249s - loss: 3.9122 - acc: 0.0170 - val_loss: 3.9121 - val_acc: 0.0178


In [ ]:
nb_epoch = 10
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 3/12
265/265 [==============================] - 271s - loss: 3.9288 - acc: 0.0128 - val_loss: 3.9121 - val_acc: 0.0200
Epoch 4/12
265/265 [==============================] - 248s - loss: 3.9142 - acc: 0.0162 - val_loss: 3.9120 - val_acc: 0.0200
Epoch 5/12
265/265 [==============================] - 249s - loss: 3.9143 - acc: 0.0155 - val_loss: 3.9120 - val_acc: 0.0200
Epoch 6/12
265/265 [==============================] - 268s - loss: 3.9141 - acc: 0.0151 - val_loss: 3.9120 - val_acc: 0.0215
Epoch 7/12
265/265 [==============================] - 249s - loss: 3.9135 - acc: 0.0143 - val_loss: 3.9123 - val_acc: 0.0163
Epoch 8/12
264/265 [============================>.] - ETA: 0s - loss: 3.9133 - acc: 0.0140

In [ ]:
nb_epoch = 10
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

265/265 [==============================] - 249s - loss: 3.9134 - acc: 0.0132 - val_loss: 3.9119 - val_acc: 0.0193
Epoch 22/22
265/265 [==============================] - 249s - loss: 3.9132 - acc: 0.0155 - val_loss: 3.9120 - val_acc: 0.0207


In [ ]:
model.load_weights('../models/sketchnet3/latest.h5')